# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [3]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [4]:
document = ET.parse( './data/mondial_database.xml' )

In [5]:
import pandas as pd
import numpy as np

#### 1. 10 countries with lowest infant mortality rates

In [6]:
ctry_dict = {}

# Loop to find Country name and Infant Mortality rate
for element in document.iterfind('country'):
    country = element.find('name')
    infant = element.find('infant_mortality')
    
    if country != None:
        if infant != None:
            ctry_dict[country.text] = float(infant.text)
        else:
            ctry_dict[country.text] = np.NaN

# Convert to data frame from dictionary
country_df = pd.DataFrame.from_dict(ctry_dict, orient='index')

# Set Column to Infant Mortality Rate
country_df.columns = ['Infant_Mortality_Rate']
country_df.sort_values(by='Infant_Mortality_Rate', ascending=True).head(10)

,Infant_Mortality_Rate
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


#### 2. 10 cities with the largest population

In [8]:
city_list = []

# Loop to find city names and provinces
for ctry in document.iterfind('country'):
    ctry_cities = ctry.findall('city')
    provinces = ctry.findall('province')

    for province in provinces:
        ctry_cities += province.findall('city')

# Loops for city name and last population entry
    for city in ctry_cities:
        city_name = city.find('name').text
        population = city.find('.//population[last()]')
# Skips popluation count if node is empty
        if population != None:
            city_pop = int(population.text)
        cities_pop = (ctry.find('name').text, city_name, city_pop)
        city_list.append(cities_pop)

city_df = pd.DataFrame.from_records(city_list, columns=['Country', 'City', 'Population'])
city_df.sort_values('Population', ascending=False).head(10)

,Country,City,Population
1341,China,Shanghai,22315474
771,Turkey,Istanbul,13710512
1527,India,Mumbai,12442373
479,Russia,Moskva,11979529
1340,China,Beijing,11716620
2810,Brazil,São Paulo,11152344
1342,China,Tianjin,11090314
1064,China,Guangzhou,11071424
1582,India,Delhi,11034555
1067,China,Shenzhen,10358381


#### 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [7]:
# Set blank dictionary
country_pop = {}

# Loops to find country name and recent population count
for element in document.iterfind('country'):
    country = element.find('name')
    population = element.find('.//population[last()]')

# Creates dictionary with Country as Key and Population count as Value
    if country != None:
        if population != None:
            country_pop[country.text] = int(population.text)
    else:
        country_pop[country.text] = np.NaN

# Creates dataframe
ctry_df = pd.DataFrame.from_dict(country_pop, orient='index')
ctry_df.reset_index(drop=False, inplace=True)
ctry_df.columns = ['Country', 'Population']
ctry_df.head()

,Country,Population
0,Albania,2800138
1,Greece,10816286
2,Macedonia,2059794
3,Serbia,7120666
4,Montenegro,620029


In [8]:
# Creates blank list to hold ethnicity percentages
ethnicity = []

# Loops for ethnicgroup percentages and adds them to list
for country in document.iterfind('country'):
    ctry = country.find('name').text

    for ethnic in country.iterfind('ethnicgroup'):
        ethnic_name = ethnic.text
        ethnic_pct = ethnic.attrib['percentage']
        ethnicity.append([ctry, ethnic_name, ethnic_pct])

# Creates data frame from list and merges it with country popluation dataframe
ethnicity_df = pd.DataFrame(ethnicity, columns=['Country', 'Ethnicity', 'Percentage'])
ethnicity_df = ethnicity_df.merge(ctry_df[['Country', 'Population']], on='Country')
ethnicity_df.columns = ['Country', 'Ethnicity', 'Percentage', 'Ctry Population']
ethnicity_df.head()

,Country,Ethnicity,Percentage,Ctry Population
0,Albania,Albanian,95,2800138
1,Albania,Greek,3,2800138
2,Greece,Greek,93,10816286
3,Macedonia,Macedonian,64.2,2059794
4,Macedonia,Albanian,25.2,2059794


In [11]:
ethnicity_df['Ethnic Population'] = (ethnicity_df['Percentage'].astype(float) / 100) * ethnicity_df['Ctry Population']
ethnicity_df['Ethnic Population'] = ethnicity_df['Ethnic Population'].astype(int)
ethnicity_df.head()

,Country,Ethnicity,Percentage,Ctry Population,Ethnic Population
0,Albania,Albanian,95,2800138,2660131
1,Albania,Greek,3,2800138,84004
2,Greece,Greek,93,10816286,10059145
3,Macedonia,Macedonian,64.2,2059794,1322387
4,Macedonia,Albanian,25.2,2059794,519068


In [12]:
eth_pop_df = ethnicity_df.groupby(['Ethnicity'])['Ethnic Population'].apply(sum).reset_index()
eth_pop_df.sort_values('Ethnic Population', ascending = False, inplace=True)
eth_pop_df.head(10)

,Ethnicity,Ethnic Population
113,Han Chinese,1245058800
120,Indo-Aryan,871815583
89,European,494872201
2,African,318325104
77,Dravidian,302713744
176,Mestizo,157734349
42,Bengali,146776916
217,Russian,131856989
128,Japanese,126534212
163,Malay,121993548


#### 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [12]:
# Create Country_code / Country dataframe
ctry_dict = {}

# Fills dictionary with country code ('car code') and country name
for code in document.iterfind('country'):
    ctry_dict[code.attrib['car_code']] = code.find('name').text

ctry_codes_df = pd.DataFrame.from_dict(ctry_dict, orient='index')
ctry_codes_df.reset_index(drop=False, inplace=True)
ctry_codes_df.columns = ['Country_Code', 'Country']
ctry_codes_df.head()

,Country_Code,Country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro


##### a) longest river

In [17]:
river_list = []

# Loops for river names, lengths, and country codes
for river in document.iterfind('river'):
    river_name = river.find('name').text
    river_length_element = river.find('length')
    if river_length_element != None:
        river_length = river_length_element.text
# Rivers run though multiple countries; codes must be split after all are selected
        for river_ctry_code in river.attrib['country'].split():
            river_ctry = ctry_dict[river_ctry_code]
            river_list.append([river_ctry, river_ctry_code, river_name, float(river_length)])
#print(river_list)
rivers_df = pd.DataFrame(river_list, columns=['Country', 'Code', 'RiverName', 'RiverLength'])
rivers_df.sort_values('RiverLength', ascending = False, inplace=True)
#rivers_df[rivers_df['Country'] == 'Brazil']
rivers_df.head(1)

,Country,Code,RiverName,RiverLength
300,Peru,PE,Amazonas,6448.0


In [14]:
rivers_df[rivers_df['RiverName'] == 'Amazonas']

,Country,Code,RiverName,RiverLength
300,Peru,PE,Amazonas,6448.0
298,Colombia,CO,Amazonas,6448.0
299,Brazil,BR,Amazonas,6448.0


##### b) largest lake

In [16]:
# Repeat for lakes
lake_list = []

for lake in document.iterfind('lake'):
    lake_name = lake.find('name').text
    lake_area_element = lake.find('area')
    if lake_area_element != None:
        lake_area = lake_area_element.text
        for lake_ctry_code in lake.attrib['country'].split():
            lake_ctry = ctry_dict[lake_ctry_code]
            lake_list.append([lake_ctry, lake_ctry_code, lake_name, float(lake_area)])
#print(lake_list)
lakes_df = pd.DataFrame(lake_list, columns=['Country', 'Code', 'LakeName', 'LakeArea'])
lakes_df.sort_values('LakeArea', ascending=False, inplace=True)
lakes_df.head(1)

,Country,Code,LakeName,LakeArea
68,Russia,R,Caspian Sea,386400.0


In [17]:
lakes_df[lakes_df['LakeName'] == 'Caspian Sea']

,Country,Code,LakeName,LakeArea
68,Russia,R,Caspian Sea,386400.0
69,Azerbaijan,AZ,Caspian Sea,386400.0
70,Kazakhstan,KAZ,Caspian Sea,386400.0
71,Iran,IR,Caspian Sea,386400.0
72,Turkmenistan,TM,Caspian Sea,386400.0


##### c) airport at highest elevation

In [18]:
# Repeat for airports
ap_list = []

for ap in document.iterfind('airport'):
    ap_name = ap.find('name').text
    ap_ctry_code = ap.attrib['country']
    ap_ctry = ctry_dict[ap_ctry_code]
    ap_elev_element = ap.find('elevation')
    if ap_elev_element != None:
        ap_elev = ap_elev_element.text
        if ap_elev != None:
            ap_list.append([ap_ctry, ap_ctry_code, ap_name, float(ap_elev)])
#print(airpt_list)
ap_df = pd.DataFrame(ap_list, columns=['Country', 'Code', 'AirportName', 'AirportElev'])
ap_df.sort_values('AirportElev', ascending=False, inplace=True)
ap_df.head(1)

,Country,Code,AirportName,AirportElev
80,Bolivia,BOL,El Alto Intl,4063.0
